# Introduction to Data Labeling with Gretel Cloud APIs

This blueprint gives a very basic introduction on how to utilize Gretel's Cloud APIs to label data in a publish / subscribe model. In the below code we start a producer thread that simply creates a fake record and sends it to a temporary Gretel Cloud Project. 

Meanwhile, the main thread will consume labeled records from the project and just print them to stdout.

In [ ]:
%%capture

!pip install -U gretel-client faker

In [ ]:
import os
import getpass
import sys
import threading
import time

from faker import Faker

from gretel_client import get_cloud_client
from gretel_client.client import temporary_project
from gretel_client.projects import Project

API_KEY = os.getenv("GRETEL_API_KEY") or getpass.getpass("Gretel API Key:")

In [ ]:
def subscriber(project: Project):
    # By default, ``iter_records()`` will block waiting for
    # new labeled records to be ready for consumption. It will
    # fetch in batches, so if there are already records in the
    # stream, we will immediately yield the most recent 200 records.
    for rec in project.iter_records():
        yield rec


def publish(project: Project, event: threading.Event):
    fake = Faker()
    while not event.is_set():
        rec = {
            "name": fake.name(),
            "phone": fake.phone_number()
        }
        project.send(rec)
        time.sleep(2)


def start(api_key: str):
    client = get_cloud_client("api", api_key)
    
    # NOTE: When this context handler exits, the project will
    # be automatically deleted.
    with temporary_project(client) as project:
        print(f"*** Console Project URL: {project.get_console_url()} ***\n")
        print("Visit the link above while this example runs to see your records in Gretel Cloud!\n\n")
        publish_event = threading.Event()
        publish_thread = threading.Thread(target=publish, args=(project, publish_event))
        publish_thread.start()
        try:
            for rec in subscriber(project):
                print(rec)
        except KeyboardInterrupt:
            print("Shutting down...")
        finally:
            publish_event.set()
            publish_thread.join()
    

In [ ]:
start(API_KEY)